In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [13]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/flc_all_stats_postflash.pkl')

In [14]:
postflash_data['norm_by_dur'] = (postflash_data['sig_mean']/postflash_data['flash_dur'])

In [15]:
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits')
test[0].header['FLASHDUR']
test.close()

In [16]:
pwd

'/System/Volumes/Data/user/cmartlin/postflash_2021/postflash_analysis/notebooks'

In [17]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]

In [18]:
low_sub_pf_A_lvl_3_2021  = low_sub_pf_A_lvl_3[low_sub_pf_A_lvl_3.obsdate.str.contains("2021")]
low_sub_pf_A_lvl_3_2019  = low_sub_pf_A_lvl_3[low_sub_pf_A_lvl_3.obsdate.str.contains("2019")]

In [19]:
path_2021 = low_sub_pf_A_lvl_3_2021['path'][0]
print(path_2021)
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ieg815u3q/ieg815u3q_flc.fits')
print(test[0].header['FLASHDUR'])
print(test[0].header['FLASHLVL'])
test.close()

/grp/hst/wfc3u/postflash_2021/mastDownload/HST/mastDownload/HST/ieg821pyq/ieg821pyq_flc.fits
1.3
3


In [20]:
path_2019 = low_sub_pf_A_lvl_3_2019['path'][0]
print(path_2019)
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/idus27w8q/idus27w8q_flc.fits')
print(test[0].header['FLASHDUR'])
print(test[0].header['FLASHLVL'])
test.close()

/grp/hst/wfc3u/postflash_2021/mastDownload/HST/mastDownload/HST/idus27w8q/idus27w8q_flc.fits
1.2
3


In [21]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]

In [22]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['norm_by_dur'])
y = low_sub_pf_A_lvl_3['norm_by_dur']/low_sub_pf_A_lvl_3['norm_by_dur'][first_rootname]

print(mean_mean)

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
#p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

1.6401700672621566
-2.6069671772532874e-06


In [23]:
low_sub_pf_A_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 12)]



# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['norm_by_dur'])
y = low_sub_pf_B_lvl_12['norm_by_dur']/low_sub_pf_B_lvl_12['norm_by_dur'][first_rootname]

print(mean_mean)

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
#p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, legend_label = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

1.4997261023925068
Slope:  -5.760126452695386e-06
